# 0. Initialize

## 0.1. Import Libraries

In [12]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os, sys, glob
import gzip
import random
import tqdm
import json
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

from IPython import display
import matplotlib as mpl
from matplotlib import pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 0.2. DEFINE VARIABLES 

In [13]:
DATA_PATH = 'data/' # '<insert-your-training-data-path-here>'

ROUND = 3 # This project will have 3 rounds of predictions: 1,2,3
STUDENT_ID = '28029'#'<insert-your-id-here>'
PROJECT_CODE = 'CS41277697ca80d70'#'<insert-your-code-here>' # Same code for the annotation eg. CS412xxxxx

## 0.3. Read Training & Evaluation Data

### 0.3.1. Get the labels for tweets

In [14]:
#trainingTweetDf = pd.read_csv('{}training-tweet.csv'.format(DATA_PATH))
trainingTweetDf = pd.read_csv('{}training-tweet.csv'.format(DATA_PATH), dtype={'tweet_id': str, 'isPolitical': str})
trainingTweetDf['isPolitical'] = trainingTweetDf['isPolitical'].apply(lambda x: 1 if x=='Yes' else 0)
trainingTweetDf

,tweet_id,isPolitical
0,1597170281545551872,1
1,1431700027471192069,0
2,1566035577090281472,1
3,1591538690869940225,1
4,1583898169238167554,1
...,...,...
2995,1593539327623151619,1
2996,1393886554062524418,0
2997,1597925615092764672,1
2998,1585291418616176640,1


In [15]:
trainingTweetDf.isPolitical.value_counts()

1    2003
0     997
Name: isPolitical, dtype: int64

### 0.3.2. Get the labels for users

In [16]:
trainingUserDf = pd.read_csv('{}training-user.csv'.format(DATA_PATH))
trainingUserDf['isBot'] = trainingUserDf['isBot'].apply(lambda x: 1 if x=='Yes' else 0)
trainingUserDf

,screen_name,isBot
0,koftecancaddy,0
1,ahaber,0
2,selahat03949652,0
3,erdin06357062,0
4,bhct__necatii,0
...,...,...
2995,djblumenberg,0
2996,mel1sq,0
2997,eren_yz1,1
2998,ergnyildiz4,0


In [17]:
trainingUserDf.isBot.value_counts()

0    2424
1     576
Name: isBot, dtype: int64

### 0.3.3. Expand your dataset with metadata and tweets

In [18]:
# You can also expand training data by downloading your own labeled datasets following the link
# Download the documents under "Link to training data"

print('http://www.onurvarol.com/Annotation-CS412-202201/reports/report_{}.html'.format(PROJECT_CODE))

http://www.onurvarol.com/Annotation-CS412-202201/reports/report_CS41277697ca80d70.html


# 1. EXTRACT FEATURES
Under *1.1. Political Tweet Detection* and *1.2. Bot Detection*, we firstly collect raw data for processing. We then combine some of them (total_interactions = num_favorites + num_retweets) or use them to extract features (whether the tweet has one of the political entities @meralaksener, @kilicdarogluk etc.).

We expect you to collect more raw data from **tweet_metadata**, **user_profiles** and **user_tweets** files by creating a function as shown in below examples such as *check_if_retweet()* and using it while iterating over data as shown under *Merge Collected Features*.

We also expect you to create new variables as much as you can from the data in order to make your predictions more accurate. For example, you may want to check:

- The tweet sources that a user frequently uses
- Whether the user is a verified account or not

...

to assess whether **a user is a bot or not** and whether **a tweet is political or not**.

In [19]:
PATH_TO_DOWNLOADED = DATA_PATH

## 1.1. Political Tweet Detection
This part stands for the feature extraction of tweets. We start with collecting the raw data from *tweet_metadata*, then use some of them to extract features.

### 1.1.1. Get Raw Data


#### 1.1.1.1. Get Tweet ID

In [20]:
def get_tweet_id(tweet_metadata_line):
    id_str = tweet_metadata_line['id_str']
    
    return id_str

#### 1.1.1.2. Get Tweet Text

In [216]:
def get_tweet_text(tweet_metadata_line):
    text = unicode_tr(tweet_metadata_line['text'].translate(str.maketrans('', '', "\"$%&()*+-/<=>[\]^`{|}~")).translate(str.maketrans(' ', ' ', ".,;:!?")))
    text = text.lower()
    
    return text

### 1.1.2. Derive Manually Crafted Features

#### 1.1.2.1. Number of political entities (keywords, mentions, hashtags, and more)

In [441]:
political_keywords = ['15 temmuz', 'abdullah başçı', 'adalet', 'adaletli', 'adem taşkaya', 'adice', 'adil', 'adli', 'af', 'affetmelidir', 'affettürkiyem', 'ahmet akın', 'ahmet davutoğlu', 'ahmet kamil erozan', 'ak parti', 'akp', 'akparti', 'ali babacan', 'ali dinçer çolak', 'ali ihsan yavuz', 'ali öztunç', 'alimahir', 'alımı', 'anlayamadiniz', 'aslan yaman', 'atama', 'atamaları', 'atamanın', 'ataması', 'atamavakti', 'atama…', 'atanmak', 'ayhan sefer üstün', 'ayhan türk', 'aysun hatipoğlu potgieter', 'ayşe acar başaran', 'ayşe keşir', 'babacan', 'bahadır bumin özarslan', 'bahadır erdem', 'bakanlık', 'bakanlığı', 'bakanı', 'bakanım', 'bakanımız', 'bartu soral', 'başkan', 'başkanlık', 'başkanlığında', 'başkanı', 'başkanım', 'başkanımız', 'başkonsolosumuz', 'bekliyoruz', 'belediye', 'belediyesi', 'bilge yılmaz', 'binali yıldırım', 'birol aydemir', 'bizimdevletimiz', 'borçlanma', 'burak akburak', 'burak dalgın', 'bülent kuşoğlu', 'bülent tezcan', 'büyüklerimiz', 'büyükşehir', 'bırakınbu', 'c kani torun', 'candan karlıtekin', 'ceza', 'cezalar', 'cezmi polat', 'chp', 'chpli', 'cumhur', 'cumhurbaskanımızın', 'cumhurbaşkanlığı', 'cumhurbaşkanı', 'cumhurbaşkanım', 'cumhurbaşkanımız', 'cumhuriyet', 'cumhuriyetimizin', 'cumhuriyetçi', 'demokrasi', 'demokrat parti', 'desteğiniz', 'deva', 'deva partisi', 'devlet', 'devlet bahçeli', 'doğa şanlıoğlu', 'doğan demir', 'ebru günay', 'edip semih yalçın', 'ediyoruz…', 'efkan ala', 'ekmeğim', 'ekonomi', 'ekpss', 'elif esen', 'enflasyon', 'erdoğan', 'erdoğanaffı', 'erkan kandemir', 'etsuriyeli', 'evrim rızvanoğlu', 'eyt', 'eyy', 'eyyüp kadir inan', 'eşitlik', 'faik öztrak', 'faikoztrak', 'fatih şahin', 'feleknas uca', 'feramuz üstün', 'feridun bilgin', 'feridun yıldız', 'fethi açıkel', 'fethi yıldız', 'fetö', 'fetöcü', 'figen yüksekdağ', 'fikret bayır', 'filiz kılıç', 'gamze akkuş ilgezdi', 'garo paylan', 'genelaf', 'gökçe gökçen', 'görüşmelerine', 'gülizar biçer karaca', 'gürol ayan', 'güvenlik', 'habibe çiftçioğlu', 'hain', 'hakaret', 'hakkı saruhan oluç', 'hakkımız', 'hakları', 'hamza dağ', 'hasan karal', 'hayati yazıcı', 'hdp', 'hdpkk', 'helalestirmekten', 'hukuk', 'hükümet', 'ibb', 'idris şahin', 'iftira', 'ihlali', 'iktidara', 'ilknur birol', 'imamoğlu', 'ismail faruk aksu', 'ismail türk', 'ismail özdemir', 'ismet büyükataman', 'istemiyoruz', 'istifa', 'istihdam', 'istiyoruz', 'istiyoruz…', 'ittifakı', 'iyi parti', 'iyiparti5yaşında', 'izzet ulvi yönter', 'işsizliğe', 'julide sarıeroğlu', 'kadro', 'kadroya', 'kamil aydın', 'kamu', 'kemal kılıçdaroğlu', 'kerim rota', 'kgm', 'khk', 'koalisyon', 'komisyonunda', 'koray aydın', 'koçak tükenmez', 'kpssden', 'kumpas', 'kılıçdaroğlu', 'lale karabıyık', 'leyla şahin usta', 'lutfuturkkan', 'lütfü şahsuvaroğlu', 'mahfuz güleryüz', 'mahkeme', 'mahkum', 'mansur', 'mağdur', 'mağduriyeti', 'mağdurları', 'mağdurlarının', 'mağduru', 'medeni yılmaz', 'mehmet ali pulcu', 'mehmet emin ekmen', 'mehmet salim ensarioğlu', 'mehmet tolga akalın', 'mehmet özhaseki', 'meral akşener', 'meral danış beştaş', 'metin ergün', 'mevlüt karakaya', 'mhp', 'milletvekili', 'mithat sancar', 'muhalefet', 'muharrem erkek', 'muharrem ince', 'muhattap', 'murat yıldız', 'musa piroğlu', 'mustafa can', 'mustafa ergen', 'mustafa gözel', 'mustafa kalaycı', 'mustafa mente', 'mustafa nedim yamalı', 'mustafa uçak', 'mustafa yeneroğlu', 'mustafa şen', 'mustafadestici', 'müdürlüğüne', 'müjde', 'müjdeli', 'müjdesi', 'münevver helün fırat', 'naci sönmez', 'nazlı seda vural', 'neferleri', 'numan kurtulmuş', 'nurettin canikli', 'onursal adıgüzel', 'oy', 'oğuz kaan salıcı', 'parlementer', 'parti', 'partisi', 'pelin yıkık', 'pençekilit', 'pervin buldan', 'pkk', 'recep', 'recep tayyip erdoğan', 'reis', 'reisimiz', 'rıdvan uz', 'sadir durmaz', 'sadullah ergin', 'salih zeki altun', 'salim kaplan', 'samet mengüç', 'sanem oktar', 'sayın', 'sağlık', 'sedat timur', 'selahattin demirtaş', 'selim temurci', 'selin sayek böke', 'selçuk özdağ', 'sema silkin ün', 'seninleyiz', 'serap yazıcı', 'serkan özcan', 'sesimiz', 'sesimize', 'sesimizi', 'sevda gül tunçer', 'sevda özbek', 'sevdamız', 'sevdası', 'sevdasına', 'sevinç atabay', 'seyit torun', 'seyit yücel', 'seçim', 'seçimöncesi', "sgk'ye", 'siyaset', 'siyaseti', 'siyasi', 'sn', 'sn.', 'soylu', 'soysuz', 'staj', 'sultan özcan', 'sözleşmeli', 'sözleşmelisi', 'sülooo', 'sülüüüü', 'talebimiz', 'tamer osmanağaoğlu', 'tayip temel', 'tayyip', 'taşeron', 'tbmm', 'tbmm’de', 'tehdit', 'terör', 'terörist', 'teşkilat', 'tuncer bakırhan', 'tuğrul kihtir', 'tuğrul çağrı kayalak', 'türkiyem', 'türkiyemizin', 'unutmayın', 'uyusturucudanda', 'uğur batur', 'uğur poyraz', 'vedat demiröz', 'vekil', 'vekilim', 'veli ağbaba', 'veli saçılık', 'veliagbaba', 'yakalım', 'yasemin bilgel', 'yazıklar', 'yaşar yıldırım', 'yunus parım', 'yüksel taşkın', 'yüzüncüyıla', 'yüzüncüyılda', 'yıkım', 'zam', 'çiftçilerimizle', 'çiğdem karaaslan', 'ömer ileri', 'ömer çelik', 'örgütü', 'örgütünün', 'özdağ', 'özgürlük', 'özlem zengin', 'ümit dede', 'ümit dikbayır', 'ümit yardım', 'ümit özdağ', 'ümit özlale', 'ünzile yüksel', 'şakşak', 'şaziye köse', 'şenol sunat']
political_mentions = ['06melihgokcek', '55erhanusta', 'LutfuTurkkan', '_aliyalcin_', '_cevdetyilmaz', '_fatihsahin', 'abdulhamitgul', 'abdullahbasci', 'adalet_bakanlik', 'adem__taskaya', 'adem_taskaya', 'aefakibaba', 'ahmet_davutoglu', 'ahmet_erozan', 'ahmetedip_ugur', 'akaraismailoglu', 'akbasogluemin', 'akburak', 'akgencistanbul', 'akgenclikgm', 'akgencmalatya', 'akkadingm', 'akkadın_malatya', 'akparti', 'akpartiistanbul', 'akpartimalatya', 'ali_oztunc', 'alibabacan', 'alidincercolak', 'aliihsanyavuz54', 'alimahir', 'alipulcu', 'alpayozalan35', 'arzuerdemdb', 'av_burcu03', 'av_ebrugunay', 'avabdullahguler', 'avantmen33', 'avhamzadag', 'avidrissahin', 'avorhankırcalı', 'avozlemzengin', 'avsabriozturk', 'avtolgaakalin', 'avunzileyuksel', 'ayangurol', 'ayhanbarut01', 'ayhanseferustun', 'ayhanturkz', 'ayseacar_b', 'aysekesir', 'ayyildirim1', 'bakanguler', 'bakisimsekmhp', 'barisyarkadas', 'bartusoral', 'bayirfikret', 'bbahadirerdem', 'bbbozarslan', 'bbpgenelmerkez', 'bdalgin', 'bekirpakdemirli', 'belginuygur10', 'birolaydemir15', 'birolaydinsp', 'bkusoglu', 'bmustafagozel', 'bulent_arinc', 'bulenttezcanchp', 'bulenttufenkci', 'buyukataman', 'by', 'bybekirbozdag', 'c_karlitekin', 'canan_kaftanci', 'celebimehmeta', 'cenginyurt52', 'cezmi_polat', 'chp_istanbulil', 'chpgencistanbul', 'chpgenclikgm', 'chpmuratbakan', 'ck_samsun', 'csbgovtr', 'csgbakanligi', 'csgbfarukcelik', 'dbdevletbahceli', 'deryayanikashb', 'devapartisi', 'dfikrisaglar', 'dogandemir41', 'dogasanlioglu', 'dpgultekinuysal', 'dr_demircan', 'dr_sozdemir', 'draslanyaman', 'drfahrettinkoca', 'drfahrettinkoca…', 'dromerileri', 'drsametmenguc', 'drsoysal', 'dursunatas38', 'dıbalierbas', 'e_semihyalcin', 'eczburhan', 'eczozgurozel', 'efkanala', 'egemenbagis', 'ekrem_imamoglu', 'elifesendeva', 'emin_ekmen', 'emine_gozgec16', 'emniyetankara', 'emniyetgm', 'emrullahisler', 'enginaltaychp', 'enginozkoc', 'erdoganbayraktr', 'erenerdemnet', 'erginsadullah', 'erkanakcay45', 'erkankandemir', 'eyupkadirinan', 'fahrettinaltun', 'fahrettinyokus', 'faikoztrak', 'fatihsahin', 'fatmasahin', 'fatmasahin_aspb', 'feleknasuca', 'feramuzustun', 'feridunbilgin64', 'feridunyildiz', 'fethigurer', 'figenyuksekdag', 'fuatoktay', 'gamzeilgezdi', 'garopaylan', 'gazianteppolis', 'gergerliogluof', 'gokcegokcen', 'gokcegokcen_', 'guleryuzmahfuz', 'gulizarbicer', 'gurseltekin34', 'hakan_cavusoglu', 'hasandogan', 'hasankaraltr', 'hatipoglupaysun', 'hayati_yazici', 'hbb_ciftcioglu', 'hdpdemirtas', 'hdpgenelmerkezi', 'helunfirat', 'herkesicinchp', 'ifarukaksu', 'ikalin1', 'ikoncuk', 'ilknur_birol', 'ismail_turk55', 'istanbul_egm', 'iyiparti', 'jsarieroglu', 'kadir_topbas', 'kamilaydinmhp', 'kaptmustafacan', 'karakayamevlut', 'kartalakgenclik', 'kasapoglu', 'kerimrota', 'kilicdarogluk', 'kilicdarogluk…', 'kocaeliemniyeti', 'kocaktukenmez', 'korayaydintr', 'lalekarabiyik', 'leylasahinusta', 'lutfusahsuvar', 'lutfuturkkan', 'mahirunal', 'mansuryavas06', 'matillakaya', 'medeniylmz', 'mehmedmus', 'mehmetersoy57', 'mehmetnuriersoy', 'mehmetozhaseki', 'memetsimsek', 'meral_aksener', 'meraldanis', 'metinergun48', 'mevlutcavusoglu', 'mhp_bilgi', 'mhptbmmgrubu', 'mithatsancarr', 'mkulunk', 'mnedimyamali', 'mstanrikulu', 'mt_goksu', 'muezzinogludr', 'muharremerkek17', 'murat_kurum', 'muratozgur2001', 'musaproglu', 'mustafa_destici', 'mustafaergen_', 'mustafamente', 'mustafasentop', 'mustafaucak07', 'mvhuseyinyildiz', 'myeneroglu', 'mzyensevkin', 'nacicinisli', 'nacicinisli…', 'nazlisedavural', 'numankurtulmus', 'nureddinnebati', 'nurettincanikli', 'oguzksalici', 'olcaykilavuz', 'omerrcelik', 'onursaladiguzel', 'orhanssumer', 'osmannnurika', 'pekcan', 'pervinbuldan', 'prfdrfilizkilic', 'profbilgeyilmaz', 'ramazancan0071', 'ridvanuz17', 'rizvanogluevrim', 'rterdogan', 's_turpcu', 'saadethukuk', 'saadetpartisi', 'sadirdurmaz', 'sagaltun', 'saglikbakanligi', 'salimensarioglu', 'samiltayyar27', 'sanemoktar', 'saruhanoluc', 'saziyekose59', 'sedattimur161', 'seferaycan', 'selcukozdag', 'selimtemurci', 'selinsayekboke', 'semasilkin', 'senolsunat', 'serapprof', 'serkanozcan1', 'serkantopalchp', 'sevdagultuncer', 'sevincatabay', 'seyit_torun', 'seyityucel06', 'sezaitemelli', 'sgirgin48tbmm', 'sonmeznaci', 'stajmadurlarıd1', 'suatkilic', 'suleymansoylu', 'suleymansoyluhayati_yazici', 'sultanozcan38', 't_karamollaoglu', 'tanjuozcanchp', 'tayiptemel', 'tbmmresmi', 'tc_disisleri', 'tc_icisleri', 'tc_icisleri…', 'tcaytunciray', 'tcbestepe', 'tckayalak', 'tcmeb', 'tcsavunma', 'tmrosmanagaoglu', 'torunkani', 'tugrulk_zaferp', 'tuncerbakirhan', 'tuncsoyer', 'turanbulent', 'ugur__poyraz', 'ugur_poyraz', 'ugurbaturr', 'ulviyonter', 'umitdikbayir', 'umitozdag', 'umitozlale', 'umityardim1961', 'umutdede16', 'vahitkirisci', 'varank', 'vedatbilgn', 'vedatbilgn…', 'vedatdemiroz', 'vekilince', 'veliagbaba', 'velisacilik', 'veyseleroglu', 'yasaryldrmmhp', 'yaseminbilgel1', 'yavuzagiraliog', 'yildirimkaya40', 'yildizfeti', 'yilmaz_ismet58', 'yilmaztunc', 'yukseltaskinn', 'yunusparim', 'yunusparım', 'zaferpartisi', 'ısliyendr']
political_hashtags = ['100yılaffı', '10aralıkdünyainsanhaklarıgünü', '10binkadro', '112açmyeacilçmhalımı', '112açmyeortaksızalım', '112açmyeçmhdenalım', '112açmyeçmhyakışır', '112açmçmhiledahagüçlü', '112açmçmhi̇ledahagüçlü', '112çmhyetektercihyetersiz', '120bintaşeronbekliyor', '15binziraatmilliekonomiye', '2000sonrasısgklıadaletistiyor', '2022kpssiptaledilsin', '2023yolundabiriz', '21yüzyıldataşeronolmasın', '5binatamaradyolojiye', 'adaletaffı', 'amerikanınçocuklarısınız', 'aralıksonuşartsızeyt', 'aralıktakadroocaktaeş', 'askerlikborclanması', 'atamayapmaönlisansıbekle', 'bagkurtescilimiyoksayma', 'bagkurtescilmagdurlari', 'bağkurtescilmağdurları', 'belediyeşirketikadroya', 'benimoyum100binatamaya', 'bütcedetaşeronvarmı', 'bütçeden100binöğretmen', 'cb15binziraatmüh', 'cbingilizce10binatama', 'cbsağlıkyönetimibeşbinkadro', 'cekehapiscezasıkalksın', 'cumhurücretliögrtkadro', 'cumhurücretliöğrtkadro', 'denklikaffımeclisegelsin', 'devlettetaşeronolurmu', 'dib15binalımhakkımız', 'diyanet4300yedekmuejdebekliyor', 'diyetisyen245ekarşı', 'ehliyetaffi', 'ehliyetaffı', 'elektrikelektronikmüh3binkadro', 'emekleriyasalaratakılanlar', 'emeklilikteyaşatakılaniar', 'emeklilikteyaşatakılanlar', 'emeklilkteyasatakılanlar', 'emlakveemlakyönetimi', 'engelliyeatamamüjdesi', 'erdoğandan100binöğretmen', 'eyt', 'eyt99sonuhakkımız', 'eyt99sonuolsun', 'eytdepremmağdurları', 'eytdeyaşkoşulunahayır', 'eythakkınıtami̇stiyor', 'eytlininsabrıtaştı', 'fahriöğreticilerekadroşart', 'geleceğizyinegeleceğiz', 'geliyorgelmekteolan', 'genelaf', 'genelafsebepleri', 'güvenlikkorucularınsesiniduyun', 'haritakadastroteknikerleriatanamıyor', 'hbystaşeronkalmasın', 'herkesekadrouzmçvşayokmu', 'ikinciyüzyılaçağrı', 'imaraffıistiyoruz', 'imarbarışıgüncellensin', 'insaatmühendisleriatamabekliyor', 'inşaatmühendislerikamuya', 'iyiolankazansın', 'iyiparti5yaşında', 'içişleriyedekleratansın', 'jeolojimüh1000atama', 'kabinedetaşerondaolsun', 'kabinesözlmyekadro', 'kadrodaöncelikuzmçvşa', 'kadrohaktır', 'kadroiçinmeclise', 'kadrovekili̇mamlarınhakkı', 'kahramanlarakadroverin', 'kamudakadrosuzkalmasın', 'kamudışıkadroya', 'kamui̇şçisikadrosuzkalmasın', 'kamuya5000mimar', 'kamuyagıdamühşart', 'kapımagelmeoyi̇çin', 'kasapogludestekver', 'kasıma100binöğretmen', 'kasımdaadil100binatama', 'kgm1000atama', 'kgmmüşavirekadrohaktır', 'kimlertaşeronişçisi', 'kislasizmakulbedelli', 'kurtulmakistiyoruz', 'köyokullarinakadroluata', 'kılıçdaroğlunesöyledi', 'kıslasızmakulbedelli', 'laboratuvaraadilatama', 'lisetıbbisekretere0kadro', 'mebdenedebiyata10bin', 'meblisegeneliaf', 'meclisteteröristi̇stemiyoruz', 'memurögrtverimliistihdam', 'muhalefetstajıkonuş', 'müjdezamanıgelmedimi', 'ogretmenlersusmayacak', 'onlisanspomeme15bin', 'oyum2020kpssye', 'patronsağlıkbakanıi̇stemiyoruz', 'pomem15binistiyor', 'pomeme15binkontenjan', 'pomemeyedekcagri', 'puançokkadroyok', 'sansüryasasınahayır', 'sartlıeytyehayır', 'sağlısollutokatlasoylu', 'sağlığavaröğretmeneyok', 'secime15binziraatmüh', 'seçime100binöğretmen', 'sicilaffı', 'sonsözüeytsöyler', 'stajaadalet', 'stajmeclisetakıldı', 'stajsskbaslangiciolsun', 'stajsskbaslangıcıbekliyor', 'stajsskbaşlangıcıolsun', 'stajyerinselamıvar', 'stajyermuhalefetdiyor', 'stajyoksaoydayok', 'sözleşmelimemuraşartsızkadro', 'sıradastajolsun', 'takkedüştükelgörüldü', 'tarikatvecemaatlerkapatilsin', 'tarikatvecemaatlerkapatılsın', 'taseronucubebirsistemdir', 'taşeronaacilçözüm', 'taşeronahayırkadroyaevet', 'taşeronaşartsızkadro', 'taşerondaadaletliolun', 'taşeronmeclisegelmeli', 'taşeronnedenbekliyor', 'taşeronradyolojikadrobekliyor', 'taşeronradyolojiyekadro', 'taşeronsuztürkiyei̇stiyoruz', 'taşerontorbaya', 'taşeronucubebirsistemdir', 'taşeronunsabrıbitti', 'taşeronunsabrıkalmadı', 'tbmm', 'tbmmvekilimamlarakadro', 'tcyüzyılıücretliögrtkadro', 'torbayasadayhs', 'tümtaşeronaayrımsızkadro', 'tümtaşeronlarakadro', 'türkiyeyüzyılı', 'uzmançavuşlarakadro', 'uzmançavuşlarkadroya', 'uzmçvşabayramolsun', 'uzmçvşlarınsözleşmeçilesi', 'vahitkirisci15binziraatmüh', 'vatansavunanakadro', 'vergideadaletbelediyedekadro', 'verilensözlertutulsunartık', 'veterinerhekimler5000atamaistiyor', 'yabancıuyrukluasistanhekimler', 'yanındayızimamoğlu', 'yhskaldirilsin', 'yüzbinöğretmeninsesi', 'yüzbinöğretmenmecliste', 'yüzyılınataması24kasımda', 'yüzüncüyıla100binatama', 'ziraate15bin6limasa', 'ziraatmüh15binatama', 'zorunluemeklilikkaldırılsın', 'zorunluemeklilikzulümdür', 'ögretmeneenaz25bin', 'önlisanssosyalgüvenlik2binatama', 'önlisanssosyalgüvenlik4binatama', 'özdağadokunma', 'özelleştirme4blileriunvaniylakadroya', 'öğretmeneenaz25bin', 'öğretmeneyüzde100zam', 'ümitözdağ', 'şanınayakışır', 'şartlıeytyehayır', 'şartsızkadro', 'şehadetesaskadrohaktır', 'şehirhastanelerinekadro']
net_politicals = ['ak parti', 'chp', 'iyi parti', 'seçim', 'devlet', 'süleyman soylu', 'atama', 'meral akşener', 'kemal kılıçdaroğlu', 'kılıçdaroğlu', 'ali babacan', 'fahrettin koca', 'ümit özdağ', 'devlet bahçeli', 'mansur yavaş', 'ekrem imamoğlu', 'taşeron', 'mahkum', 'khk', 'darbe', ' af ', 'adalet', 'hukuk', 'zam', 'tcmb', 'tbmm', 'vekil', 'başkan', 'çavuş', 'asker', 'polis', 'öğretmen', 'engelli', 'terör', 'vatan']
no_politicals = ['galatasaray', 'fenerbahçe', 'fenerbahce', 'besiktas', 'beşiktaş', 'trabzonspor', 'başakşehir', 'basaksehir', 'ali koç', 'mesut özil', 'muğlaspor', 'ankaragücü', 'ankaragucu']

from unicode_tr import unicode_tr
    
def get_num_pol_entities(line):
    
    num_pol_keywords = 0
    num_pol_mentions = 0
    num_pol_hashtags = 0
    erdogan_tweet = 0
    
    not_corrected_line = unicode_tr(line['text'].translate(str.maketrans('', '', "\"$%&()*+-/<=>[\]^`{|}~")).translate(str.maketrans(' ', ' ', ".,;:!?")))
    corrected_line = not_corrected_line.lower()
    
    # print(corrected_line)
    
    pol_keywords = [ent for ent in political_keywords if ent in corrected_line]
    pol_mentions = [ent for ent in political_mentions if ent in corrected_line]
    pol_hashtags = [ent for ent in political_hashtags if ent in corrected_line]
    
    num_pol_keywords = len(pol_keywords)
    num_pol_mentions = len(pol_mentions)
    num_pol_hashtags = len(pol_hashtags)
    
    net_political_list = [ent for ent in net_politicals if ent in corrected_line]
    if len(net_political_list) > 0:
        net_political = 1
    else:
        net_political = 0
        
    no_political_list = [ent for ent in no_politicals if ent in corrected_line]
    if len(no_political_list) > 0:
        no_political = 1
    else:
        no_political = 0
        
    return num_pol_keywords, num_pol_mentions, num_pol_hashtags, net_political, no_political


#### 1.1.2.2. Get possible predicted value

In [442]:
def get_predicted_political(num_pol_keywords, num_pol_hashtags, num_pol_mentions):
    if num_pol_keywords > 0 or num_pol_hashtags > 0 or num_pol_mentions > 0:
        return 1
    return 0

### 1.1.3. Collect data using the functions above and transform into a Pandas DataFrame

In [443]:
dfPolitical = {'tweet_id':[],
              'text':[],
              'num_pol_keywords': [],
              'num_pol_keywords_bin': [],
              'num_pol_mentions': [],
              'num_pol_mentions_bin': [],
              'num_pol_hashtags': [],
              'num_pol_hashtags_bin': [],
              'net_political': [],
              'no_political': [],
              'tweet_link': [],
              'predicted_political': []}
i = 0

with gzip.open(f"{PATH_TO_DOWNLOADED}tweet_metadata.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)
            
        tweet_id = get_tweet_id(line)
        text = get_tweet_text(line)
        num_pol_keywords, num_pol_mentions, num_pol_hashtags, net_political, no_political = get_num_pol_entities(line)
        num_pol_keywords_bin = 1 if num_pol_keywords >= 1 else 0
        num_pol_hashtags_bin = 1 if num_pol_hashtags >= 1 else 0
        num_pol_mentions_bin = 1 if num_pol_mentions >= 1 else 0
        predicted_political = get_predicted_political(num_pol_keywords, num_pol_hashtags, num_pol_mentions)

        tweet_url = 'https://twitter.com/' + str(line['user']['screen_name']) + '/status/' + str(tweet_id)

        dfPolitical['tweet_id'].append(tweet_id)
        dfPolitical['text'].append(text)
        dfPolitical['num_pol_keywords'].append(num_pol_keywords)
        dfPolitical['num_pol_keywords_bin'].append(num_pol_keywords_bin)
        dfPolitical['num_pol_mentions'].append(num_pol_mentions)
        dfPolitical['num_pol_mentions_bin'].append(num_pol_mentions_bin)
        dfPolitical['num_pol_hashtags'].append(num_pol_hashtags)
        dfPolitical['num_pol_hashtags_bin'].append(num_pol_hashtags_bin)
        dfPolitical['net_political'].append(net_political)
        dfPolitical['no_political'].append(no_political)
        dfPolitical['tweet_link'].append(tweet_url)
        dfPolitical['predicted_political'].append(predicted_political)

In [444]:
dfPolitical = pd.DataFrame(dfPolitical)
dfPolitical

,tweet_id,text,num_pol_keywords,num_pol_keywords_bin,num_pol_mentions,num_pol_mentions_bin,num_pol_hashtags,num_pol_hashtags_bin,net_political,no_political,tweet_link,predicted_political
0,1588568792984346624,sosyal hizmetin temelini çocuk oluştururçocuğu...,0,0,0,0,0,0,0,0,https://twitter.com/Maviruh_/status/1588568792...,0
1,1588452263047069697,@mahirunal gavur izmir ya onlar hani cumhuriye...,2,1,1,1,0,0,0,0,https://twitter.com/mtfdan/status/158845226304...,1
2,1569589330544398336,#şehitadayıuzmçvşakadro\nsiz istesenizde ıstem...,1,1,0,0,0,0,0,0,https://twitter.com/AhSuCilginUzman/status/156...,1
3,1570428119609139201,@ajans_muhbir siz kaypak olmayıp onay vermesey...,0,0,0,0,0,0,0,0,https://twitter.com/HamitElKelle/status/157042...,0
4,1551163840368414722,engelli öğretmenler olarak önümüzdeki engeller...,0,0,0,0,0,0,1,0,https://twitter.com/SED58417690/status/1551163...,0
...,...,...,...,...,...,...,...,...,...,...,...,...
33528,1568595408233832448,gerçek kimlik taşımayan hesaplara cevap vermem...,0,0,0,0,0,0,0,0,https://twitter.com/ardanzenturk/status/156859...,0
33529,1584027427696959488,@umitozdag neden suriyelilerle ilgili bu kadar...,0,0,1,1,0,0,1,0,https://twitter.com/ozgul_61/status/1584027427...,1
33530,1585945783307730945,@celebimehmeta niye türkiye yüzyılıdatürkiye c...,2,1,1,1,0,0,0,0,https://twitter.com/ladrekova/status/158594578...,1
33531,1569748909521801221,rt @muazzezeralp @doan58213655 @denizkonur @na...,3,1,3,1,1,1,0,0,https://twitter.com/YapiKYTgriVRLSN/status/156...,1


## 1.2. From Users

### 1.2.1. Get user metadata from user_profiles.jsons.gz

#### 1.2.1.1. Get user info metadata

In [452]:
def get_user_info_metadata(user_metadata_line):
    
    user_id = user_metadata_line['id_str']
    user_screen_name = user_metadata_line['screen_name'].lower()
    user_followers_count = user_metadata_line['followers_count']
    user_friends_count = user_metadata_line['friends_count']
    
    dictionary = {'user_id':user_id, 'user_screen_name':user_screen_name,
                  'followers':user_followers_count, 'following':user_friends_count}

    return dictionary

#### 1.2.1.2. Get followers ratio

In [453]:
def get_followers_ratio(user_followers_count, user_friends_count):
    try:
        followers_ratio =  user_followers_count / (user_friends_count + user_followers_count)
    except ZeroDivisionError:
        followers_ratio = 0
        
    return followers_ratio

#### 1.2.1.3. Get following ratio

In [454]:
def get_following_ratio(user_followers_count, user_friends_count):
    try:
        following_ratio =  user_friends_count / (user_friends_count + user_followers_count)
    except ZeroDivisionError:
        following_ratio = 0
        
    return following_ratio

#### 1.2.1.4. Get follow similarity (followers vs. following)

In [455]:
def get_follow_similarity(user_followers_count, user_friends_count):
    try:
        difference = abs(user_friends_count - user_followers_count) / (user_friends_count + user_followers_count)
    except ZeroDivisionError:
        difference = 0
    
    return (1 - difference)

#### 1.2.1.5. Get follow treshold status

In [456]:
def get_follow_threshold(follow_treshold, user_followers_count, user_friends_count):
    if (user_followers_count >= follow_treshold) or (user_friends_count >= follow_treshold):
        return 1
    else: 
        return 0

### 1.2.2. Collect data using the functions above and transform into a Pandas DataFrame

In [457]:
dfBot = {'user_id':[],
         'user_screen_name':[],
         'followers':[],
         'following':[],
         'followers_ratio':[],
         'following_ratio':[],
         'follow_similarity':[],
         'follow_threshold':[]}

with gzip.open(f"{PATH_TO_DOWNLOADED}user_profiles.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)

        dictionary = get_user_info_metadata(line)
        for k,v in dictionary.items():
            dfBot[k].append(v)
        
        followers_ratio = get_followers_ratio(dictionary['followers'], dictionary['following'])
        dfBot['followers_ratio'].append(followers_ratio)
        
        following_ratio = get_following_ratio(dictionary['followers'], dictionary['following'])
        dfBot['following_ratio'].append(following_ratio)
        
        follow_similarity = get_follow_similarity(dictionary['followers'], dictionary['following'])
        dfBot['follow_similarity'].append(follow_similarity)
        
        follow_threshold = get_follow_threshold(2000, dictionary['followers'], dictionary['following'])
        dfBot['follow_threshold'].append(follow_threshold)


In [458]:
dfBot = pd.DataFrame(dfBot)
dfBot.head(50)

,user_id,user_screen_name,followers,following,followers_ratio,following_ratio,follow_similarity,follow_threshold
0,1431241870848450577,nasreenakhan006,65,185,0.260000,0.740000,0.520000,0
1,1304340303080386560,scorpiehoez,8235,3011,0.732260,0.267740,0.535479,1
2,1116042038577958914,yusufak63712920,95,399,0.192308,0.807692,0.384615,0
3,4859899931,nedenburdaysam,40,83,0.325203,0.674797,0.650407,0
4,2225373636,biologselim,100,98,0.505051,0.494949,0.989899,0
5,1374314782669606921,aytaasl07787580,1486,1566,0.486894,0.513106,0.973788,0
6,4752895589,delidemir33,212,136,0.609195,0.390805,0.781609,0
7,1249651167078088708,pinaraslan4558,9105,5270,0.633391,0.366609,0.733217,1
8,436894338,hseynugur,5286,4947,0.516564,0.483436,0.966872,1
9,306919165,omerakdag34,5961,0,1.000000,0.000000,0.000000,1


### 1.2.2. Get Tweet Info of Users in user_profiles.jsons.gz

#### 1.2.2.1. Check ratio of retweets to all tweets

In [459]:
def get_retweet_ratio(line):
    number_retweets = 0
    number_original_tweets = 0

    for tweet in line['tweets']:
        try:
            tweet['retweeted_status']
            number_retweets += 1
                
        except:
            number_original_tweets += 1
            
    total_tweets = number_retweets + number_original_tweets
    
    if total_tweets == 0:
        retweet_total_ratio = None
    else:
        retweet_total_ratio = number_retweets/(total_tweets)
    
    return retweet_total_ratio

#### 1.2.2.2. Check ratio of tweets having media to all tweets

In [460]:
def get_not_media_ratio(line):
    not_media_ratio = 0
    num_of_tweets = len(line['tweets'])
    
    if num_of_tweets != 0:
        tweets_having_media = 0
        
        for tweet in line['tweets']:
            if 'media' in tweet['entities']:
                if len(tweet['entities']['media']) != 0:
                    tweets_having_media += 1
                
        not_media_ratio = (num_of_tweets - tweets_having_media) / num_of_tweets
            
    return not_media_ratio

#### 1.2.2.3. Get hashtag-based tweets ratio

In [461]:
def get_hashtag_tweet_ratio(threshold, line):
    
    hashtag_tweet_ratio = 0
    num_of_tweets = len(line['tweets'])
    
    if num_of_tweets != 0:
        
        hashtag_tweet_count = 0
        for tweet in line['tweets']:
            num_of_words = len(tweet['text'].split())
            
            try:
                num_of_hashtags = len(tweet['entities']['hashtags'])
            except:
                num_of_hashtags = 0
            
            if (num_of_hashtags / num_of_words) >= threshold:
                hashtag_tweet_count += 1
            
        hashtag_tweet_ratio = hashtag_tweet_count / num_of_tweets
            
    return hashtag_tweet_ratio

#### 1.2.2.4. Get average hashtag of all tweets

In [462]:
def get_avg_hashtag(line):
    
    avg_hashtag = 0
    num_of_tweets = len(line['tweets'])
    
    if num_of_tweets != 0:
        
        total_words = 0
        total_hashtags = 0
        for tweet in line['tweets']:
            num_of_words = len(tweet['text'].split())
            total_words += num_of_words
            
            try:
                num_of_hashtags = len(tweet['entities']['hashtags'])
            except:
                num_of_hashtags = 0
            
            total_hashtags += num_of_hashtags
            
        avg_hashtag = total_hashtags / total_words
            
    return avg_hashtag

#### 1.2.2.5. Get number of copy tweets

In [463]:
from difflib import SequenceMatcher

def get_copy_tweets_amount(line):
    
    copy_tweets_amount = 0
    all_tweet_texts = []
    if len(line['tweets']) != 0:
        
        for tweet in line['tweets']:
            all_tweet_texts.append(tweet['text'])
    
    for i in range(len(all_tweet_texts)):
        tweet_text1 = all_tweet_texts[i]
        if i != len(all_tweet_texts) - 1:
            tweet_text2 = all_tweet_texts[i+1]
            if (SequenceMatcher(None, tweet_text1, tweet_text2).ratio()) >= 0.8:
                copy_tweets_amount += 1
        
    return copy_tweets_amount

#### 1.2.2.6. Get average number of favorites

In [464]:
def get_avg_favorites(line):
    avg_favorites = np.median([tweet['favorite_count'] for tweet in line['tweets']])

    return avg_favorites

### 1.2.3. Collect data using the functions above and transform into a Pandas DataFrame

In [466]:
dfBotTweets = {'user_id':[],
               'tweet_count':[],
               'retweet_ratio':[],
               'not_media_ratio':[],
               'avg_favorites':[],
               'avg_hashtag':[],
               'h_tweet_ratio':[],
               'copy_tweets':[]
              }

i = 0

with gzip.open(f"{PATH_TO_DOWNLOADED}user_tweets.jsons.gz", "rb") as f:
    for line in f:

        line = json.loads(line)

        user_id = line['user_id']
        dfBotTweets['user_id'].append(user_id)
        
        dfBotTweets['tweet_count'].append(len(line['tweets']))
        
        retweet_ratio = get_retweet_ratio(line)
        dfBotTweets['retweet_ratio'].append(retweet_ratio)
        
        not_media_percentage = get_not_media_ratio(line)
        dfBotTweets['not_media_ratio'].append(not_media_percentage)
        
        avg_favorites = get_avg_favorites(line)
        dfBotTweets['avg_favorites'].append(avg_favorites)
        
        h_tweet_ratio = get_hashtag_tweet_ratio(0.8, line)
        dfBotTweets['h_tweet_ratio'].append(h_tweet_ratio)
        
        avg_hashtag = get_avg_hashtag(line)
        dfBotTweets['avg_hashtag'].append(avg_hashtag)
        
        copy_tweets = get_copy_tweets_amount(line)
        dfBotTweets['copy_tweets'].append(copy_tweets)

        i += 1
        if i % 100 == 0:
            print(i)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [468]:
dfBotTweets = pd.DataFrame(dfBotTweets)
dfBotTweets

,user_id,tweet_count,retweet_ratio,not_media_ratio,avg_favorites,avg_hashtag,h_tweet_ratio,copy_tweets
0,594642154,200,0.115000,0.975000,2.0,0.000000,0.000000,1
1,525600289,199,0.005025,0.949749,1.0,0.018315,0.005025,1
2,931895965501534209,200,0.900000,0.955000,0.0,0.051790,0.005000,14
3,1591543462746329088,200,0.185000,0.925000,0.0,0.004554,0.000000,3
4,734801354749796352,200,1.000000,0.740000,0.0,0.006906,0.000000,0
...,...,...,...,...,...,...,...,...
28310,1591370361488252928,200,0.800000,1.000000,0.0,0.030579,0.000000,16
28311,1475272459616235525,200,0.825000,0.930000,0.0,0.013543,0.000000,0
28312,1096753792731750401,196,0.051020,0.948980,1.0,0.002356,0.000000,0
28313,1269527617687953409,200,0.095000,0.925000,2.0,0.000700,0.000000,0


### 1.2.3. Merge dfBot and dfBotTweets

In [477]:
dfBotAll = dfBot.merge(dfBotTweets, how='left')

dfBotAll[['tweet_count', 'retweet_ratio', 'not_media_ratio', 'avg_favorites', 'avg_hashtag', 'h_tweet_ratio', 'copy_tweets']] = dfBotAll[['tweet_count', 'retweet_ratio', 'not_media_ratio', 'avg_favorites', 'avg_hashtag', 'h_tweet_ratio', 'copy_tweets']].fillna(0)

dfBotAll

,user_id,user_screen_name,followers,following,followers_ratio,following_ratio,follow_similarity,follow_threshold,tweet_count,retweet_ratio,not_media_ratio,avg_favorites,avg_hashtag,h_tweet_ratio,copy_tweets
0,1431241870848450577,nasreenakhan006,65,185,0.260000,0.740000,0.520000,0,197.0,0.395939,0.761421,0.0,0.007867,0.000000,0.0
1,1304340303080386560,scorpiehoez,8235,3011,0.732260,0.267740,0.535479,1,200.0,0.125000,0.900000,0.0,0.000000,0.000000,1.0
2,1116042038577958914,yusufak63712920,95,399,0.192308,0.807692,0.384615,0,200.0,0.910000,0.965000,0.0,0.082645,0.000000,2.0
3,4859899931,nedenburdaysam,40,83,0.325203,0.674797,0.650407,0,196.0,0.015306,0.908163,1.0,0.000956,0.000000,0.0
4,2225373636,biologselim,100,98,0.505051,0.494949,0.989899,0,197.0,0.659898,0.928934,0.0,0.028500,0.005076,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29665,1320834618220781569,islam_mrsj,229,217,0.513453,0.486547,0.973094,0,200.0,0.015000,0.935000,1.0,0.004640,0.005000,2.0
29666,111074128,meldaonur,212457,5428,0.975088,0.024912,0.049824,1,199.0,0.291457,0.934673,2.0,0.007336,0.000000,1.0
29667,36946875,ali_ydm,116,141,0.451362,0.548638,0.902724,0,195.0,0.061538,0.969231,0.0,0.009506,0.000000,0.0
29668,2389587396,turkanusta,1669,3639,0.314431,0.685569,0.628862,1,200.0,0.995000,0.880000,0.0,0.002478,0.000000,1.0


# 2. TRAIN MODEL

## 2.1. Political Tweet Prediction

### 2.1.1. Merge dfPolitical data with labels

In [471]:
dfPoliticalAll_train = pd.merge(dfPolitical, trainingTweetDf)
dfPoliticalAll_train

,tweet_id,text,num_pol_keywords,num_pol_keywords_bin,num_pol_mentions,num_pol_mentions_bin,num_pol_hashtags,num_pol_hashtags_bin,net_political,no_political,tweet_link,predicted_political,isPolitical
0,1585955683513798656,@avozlemzengin yüzüncüyıla yakışangenelaf adli...,6,1,1,1,1,1,1,0,https://twitter.com/zehra78231638/status/15859...,1,1
1,1597631718479261696,#tcyüzyılıücretliögrtkadro\n#tcyüzyılıücretliö...,2,1,0,0,1,1,1,0,https://twitter.com/NurOzguler/status/15976317...,1,1
2,1572522789948751874,ekrem imamoğlu davayı değerlendirdi 'boş işler...,1,1,0,0,0,0,1,0,https://twitter.com/onediocom/status/157252278...,1,1
3,1591412481561624577,sayın bakanım @suleymansoylu pomem önlisans er...,5,1,1,1,0,0,1,0,https://twitter.com/BuckyBarnesTR/status/15914...,1,1
4,1596914274907348992,@varank sayın bakanım bodrumdaki bu araziyi n...,3,1,1,1,0,0,0,0,https://twitter.com/SayariAhmet/status/1596914...,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,1565798698029924353,@dbdevletbahceli şoför,1,1,1,1,0,0,1,0,https://twitter.com/Ahmetazrak8/status/1565798...,1,0
2996,1586019337248309248,türkiyenin geleceği ücretli öğretmenliğe emane...,0,0,2,1,0,0,1,0,https://twitter.com/taskinali003/status/158601...,1,1
2997,1589888339108585474,@enginozkoc artık yeter \nsuç duyurusudur\n@rt...,0,0,3,1,0,0,0,0,https://twitter.com/00Can02/status/15898883391...,1,1
2998,1587113229041508352,@mahirunal parti adına doğrusunu yaptın,1,1,1,1,0,0,0,0,https://twitter.com/TeomanTiryakio2/status/158...,1,1


In [472]:
dfPoliticalAll_train.corr()

/var/folders/rr/6ptn8ly52mx967t8p0kyznd40000gn/T/ipykernel_44347/2793471996.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  dfPoliticalAll_train.corr()


,num_pol_keywords,num_pol_keywords_bin,num_pol_mentions,num_pol_mentions_bin,num_pol_hashtags,num_pol_hashtags_bin,net_political,no_political,predicted_political,isPolitical
num_pol_keywords,1.000000,0.654034,0.274861,0.273803,0.169722,0.174511,0.568171,-0.044684,0.444459,0.342960
num_pol_keywords_bin,0.654034,1.000000,0.275468,0.299075,0.168692,0.172525,0.506190,-0.049565,0.679565,0.390506
num_pol_mentions,0.274861,0.275468,1.000000,0.689491,0.166934,0.172764,0.179225,-0.059658,0.466321,0.357922
num_pol_mentions_bin,0.273803,0.299075,0.689491,1.000000,0.126289,0.127025,0.176701,-0.081484,0.676326,0.499978
num_pol_hashtags,0.169722,0.168692,0.166934,0.126289,1.000000,0.959775,0.177602,-0.019812,0.137204,0.120939
num_pol_hashtags_bin,0.174511,0.172525,0.172764,0.127025,0.959775,1.000000,0.180172,-0.020642,0.142955,0.124620
net_political,0.568171,0.506190,0.179225,0.176701,0.177602,0.180172,1.000000,-0.055708,0.359154,0.314505
no_political,-0.044684,-0.049565,-0.059658,-0.081484,-0.019812,-0.020642,-0.055708,1.000000,-0.089192,-0.110518
predicted_political,0.444459,0.679565,0.466321,0.676326,0.137204,0.142955,0.359154,-0.089192,1.000000,0.570115
isPolitical,0.342960,0.390506,0.357922,0.499978,0.120939,0.124620,0.314505,-0.110518,0.570115,1.000000


### 2.1.2. Separate X and y values
We only use 3 features here to create a baseline model. However, it is not enough to get good results.

In [473]:
featuresIsPolitical = ['num_pol_keywords_bin', 'num_pol_mentions_bin', 'predicted_political']
X1 = dfPoliticalAll_train[featuresIsPolitical]
# X1 = dfPoliticalAll_train[['num_pol_keywords','num_pol_keywords_bin','num_pol_hashtags','num_pol_hashtags_bin','num_pol_mentions','num_pol_mentions_bin']]
y1 = dfPoliticalAll_train['isPolitical']

### 2.1.3. Train - validation split

In [474]:
from sklearn.model_selection import train_test_split

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.20, random_state=42)

### 2.1.4. Train the model

Here, you may use different models such as neural networks, XGBoost, AdaBoost, RandomForest, Linear Regression, Logistic Regression etc. to see which model does the best. Also, you can use grid_search_cv() or a basic for loop to optimize the hyperparameters of your model.

In [475]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

dtc_political = LogisticRegression()

dtc_political.fit(X1_train, y1_train)
predictions = dtc_political.predict_proba(X1_test)
preds = [ent[1] for ent in predictions]
mse = mean_squared_error(y1_test, preds)

print("\n\nTest MSE:", mse, '\n\n')
print("Test Predictions:", preds)



Test MSE: 0.14905831280606224 


Test Predictions: [0.6710780613951839, 0.2062367212297227, 0.2062367212297227, 0.6710780613951839, 0.8251113619253946, 0.8913851444137931, 0.8251113619253946, 0.8251113619253946, 0.2062367212297227, 0.8913851444137931, 0.6710780613951839, 0.8913851444137931, 0.2062367212297227, 0.8251113619253946, 0.2062367212297227, 0.6710780613951839, 0.2062367212297227, 0.6710780613951839, 0.8251113619253946, 0.2062367212297227, 0.2062367212297227, 0.6710780613951839, 0.8913851444137931, 0.2062367212297227, 0.2062367212297227, 0.8251113619253946, 0.8913851444137931, 0.2062367212297227, 0.8913851444137931, 0.2062367212297227, 0.8913851444137931, 0.8913851444137931, 0.2062367212297227, 0.8251113619253946, 0.8913851444137931, 0.8251113619253946, 0.6710780613951839, 0.2062367212297227, 0.8251113619253946, 0.8913851444137931, 0.6710780613951839, 0.2062367212297227, 0.8251113619253946, 0.6710780613951839, 0.8913851444137931, 0.6710780613951839, 0.8913851444137931, 0.6710

## 2.2. Bot Detection

### 2.2.1. Merge dfBotAll data with labels

In [478]:
dfBotAll.user_screen_name = dfBotAll.user_screen_name.str.lower()

In [479]:
dfBotAll_train = dfBotAll.merge(trainingUserDf, left_on='user_screen_name', right_on='screen_name')

dfBotAll_train['isBot'] = dfBotAll_train['isBot'].replace({'Yes':1, 'Not sure':0, 'No':0})
dfBotAll_train.head(50)

,user_id,user_screen_name,followers,following,followers_ratio,following_ratio,follow_similarity,follow_threshold,tweet_count,retweet_ratio,not_media_ratio,avg_favorites,avg_hashtag,h_tweet_ratio,copy_tweets,screen_name,isBot
0,1512081815292432394,sezgin953116371,46,430,0.096639,0.903361,0.193277,0,199.0,0.050251,0.914573,0.0,0.001200,0.000000,1.0,sezgin953116371,0
1,1425452291428077571,gogoadem61,14,171,0.075676,0.924324,0.151351,0,113.0,0.761062,0.982301,0.0,0.046864,0.000000,5.0,gogoadem61,0
2,328164303,dewil511,21,49,0.300000,0.700000,0.600000,0,198.0,0.010101,0.383838,0.0,0.115628,0.000000,121.0,dewil511,1
3,1343666971368431622,midnight__bird,422,260,0.618768,0.381232,0.762463,0,200.0,0.085000,0.975000,1.0,0.004986,0.000000,0.0,midnight__bird,0
4,1240932880488038400,samedpinarci,133,202,0.397015,0.602985,0.794030,0,200.0,0.780000,0.850000,0.0,0.018056,0.000000,0.0,samedpinarci,0
5,1352372158094315524,sarilaciask07,331,393,0.457182,0.542818,0.914365,0,198.0,0.212121,0.919192,1.0,0.000000,0.000000,0.0,sarilaciask07,0
6,1258068729801424897,hakan14538,324,995,0.245641,0.754359,0.491281,0,200.0,0.000000,0.905000,0.0,0.006116,0.000000,0.0,hakan14538,0
7,1438252103424688128,besiktasl_turk,61,218,0.218638,0.781362,0.437276,0,190.0,0.410526,0.826316,0.0,0.004538,0.000000,1.0,besiktasl_turk,0
8,1123152323340120065,t_a_akgn,265,355,0.427419,0.572581,0.854839,0,199.0,0.150754,0.994975,1.0,0.000000,0.000000,0.0,t_a_akgn,0
9,774378852906860544,altunay_oktay,613,547,0.528448,0.471552,0.943103,0,200.0,0.005000,0.970000,0.0,0.000000,0.000000,111.0,altunay_oktay,0


In [480]:
trainingUserDf.isBot.value_counts()

0    2424
1     576
Name: isBot, dtype: int64

In [481]:
dfBotAll_train.corr()

/var/folders/rr/6ptn8ly52mx967t8p0kyznd40000gn/T/ipykernel_44347/2991032875.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  dfBotAll_train.corr()


,followers,following,followers_ratio,following_ratio,follow_similarity,follow_threshold,tweet_count,retweet_ratio,not_media_ratio,avg_favorites,avg_hashtag,h_tweet_ratio,copy_tweets,isBot
followers,1.000000,0.251831,0.168368,-0.167381,-0.135118,0.131249,0.029403,-0.027271,-0.017725,0.218420,-0.007882,-0.006867,-0.028919,-0.003699
following,0.251831,1.000000,0.038549,-0.036872,0.101495,0.197464,0.039519,0.002244,0.000121,0.019951,-0.026155,-0.012327,-0.035497,0.005933
followers_ratio,0.168368,0.038549,1.000000,-0.982389,-0.134962,0.502053,0.145582,-0.031371,-0.054106,0.126668,0.077524,0.000357,-0.073532,-0.084849
following_ratio,-0.167381,-0.036872,-0.982389,1.000000,0.124500,-0.494883,-0.119939,0.037540,0.063391,-0.125974,-0.081479,-0.020587,0.076668,0.085961
follow_similarity,-0.135118,0.101495,-0.134962,0.124500,1.000000,-0.052541,0.097506,0.233480,0.042414,-0.106208,0.015607,-0.034388,0.094364,0.173490
follow_threshold,0.131249,0.197464,0.502053,-0.494883,-0.052541,1.000000,0.157682,-0.014590,-0.012402,0.091408,-0.053329,-0.044019,-0.176636,-0.121875
tweet_count,0.029403,0.039519,0.145582,-0.119939,0.097506,0.157682,1.000000,0.166870,0.469899,0.009620,0.033178,-0.093695,0.087271,0.055629
retweet_ratio,-0.027271,0.002244,-0.031371,0.037540,0.233480,-0.014590,0.166870,1.000000,0.074843,-0.049879,0.082906,-0.032722,-0.015994,0.298571
not_media_ratio,-0.017725,0.000121,-0.054106,0.063391,0.042414,-0.012402,0.469899,0.074843,1.000000,-0.076076,-0.039617,0.012037,0.095398,0.076405
avg_favorites,0.218420,0.019951,0.126668,-0.125974,-0.106208,0.091408,0.009620,-0.049879,-0.076076,1.000000,-0.010992,-0.003661,-0.021826,-0.033228


### 2.2.2. Separate X and y values
We use only 4 features here to create a baseline model. However, it is not enough to get good results.

In [482]:
# X = dfBotAll_train[['description_len', 'followers_to_all_ratio', 'num_median_favorites']]
# y = dfBotAll_train['isBot']

In [483]:
#my data 
X1 = dfBotAll_train[['follow_similarity', 'copy_tweets', 'avg_hashtag', 'follow_threshold', 'retweet_ratio']]
y1 = dfBotAll_train['isBot']

### 2.2.3. Train-test split

In [484]:
from sklearn.model_selection import train_test_split

'''X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)'''

'X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)'

### 2.2.4. Train the model

In [485]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

In [486]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.15, random_state=42)

In [487]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

dtc_bot = LogisticRegression()

dtc_bot.fit(X1_train, y1_train)
predictions = dtc_bot.predict_proba(X1_test)
preds = [ent[1] for ent in predictions]
mse = mean_squared_error(y1_test, preds)

print("\n\nTest MSE:", mse, '\n\n')
print("Test Predictions:", preds)



Test MSE: 0.11512092661133291 


Test Predictions: [0.195243190373043, 0.07558655238812828, 0.06869028031105448, 0.04448461454270913, 0.10511984158917752, 0.05114666943441754, 0.1212091884837739, 0.34621005240822594, 0.19538956903510507, 0.046743972230736085, 0.08305703306844567, 0.38853005766080945, 0.06992706236809801, 0.07001079160796252, 0.058943859025811646, 0.10021087895630601, 0.2909463346228143, 0.4798586172604599, 0.10727678604726117, 0.05662614268808936, 0.4396605299985936, 0.3442423905980585, 0.04237337662878278, 0.3299058278512418, 0.4347463064826362, 0.13565900743789336, 0.10828434658662732, 0.09188827566074627, 0.05956503766347604, 0.05786972373648083, 0.05749947936158364, 0.9748380507333004, 0.23262303370364013, 0.09959350921594905, 0.13890580180301085, 0.36834985454941555, 0.2652655027414293, 0.07505638726684348, 0.058991112139687864, 0.03804360429261337, 0.048594130968327466, 0.36454090927138066, 0.09712281395307475, 0.3798499568149987, 0.03424569054526639, 0.3021926

# 3. MAKE PREDICTIONS

Here, you will make predictions with the models that you have trained above.

## 3.1. Predictions for Tweets (Political or Not)

In [497]:
# read the evaluation file as follows
evaluationTweetDf = pd.read_csv('{}evaluation-round{}-tweet.csv'.format(PATH_TO_DOWNLOADED, ROUND), dtype={0: str}, header=None, names=['tweet_id'])
evaluationTweetDf = evaluationTweetDf.dropna()
evaluationTweetDf

# merge it with the political dataframe so that you can use the make predictions based on the variables
dfPolitical_test = dfPolitical.merge(evaluationTweetDf)

X = dfPolitical_test[featuresIsPolitical]
regression_political = dtc_political.predict_proba(X)

predictions_political = []
import numpy as np
from array import array

for index, row in dfPolitical_test.iterrows():
    prediction = regression_political[index]
    # print(index, num_pol_keywords_bin, num_pol_mentions_bin)
    # print(index, prediction)
        
    if row['net_political'] == 1:
        # predictions_political.append([row['tweet_link'], 1.0])
        predictions_political.append(1.0)
    else:
        if row['no_political'] == 1:
            # predictions_political.append([row['tweet_link'], 0.0])
            predictions_political.append(0.0)
        else:
            if prediction[1] < 0.25:
                # predictions_political.append([row['tweet_link'], 0.125])
                predictions_political.append(0.125)
            elif prediction[1] > 0.8:
                # predictions_political.append([row['tweet_link'], 0.925])
                predictions_political.append(0.925)
            elif prediction[1] == 0.6710780613951839:
                predictions_political.append(row['tweet_link'])
            else:
                # predictions_political.append([row['tweet_link'], prediction[1]])
                predictions_political.append(prediction[1])

print(predictions_political)

[0.125, 0.125, 1.0, 0.925, 1.0, 0.125, 'https://twitter.com/awfhdsr6/status/1559963768372740098', 'https://twitter.com/akanturker/status/1562853131251118081', 0.925, 0.925, 'https://twitter.com/venxzhan/status/1427746815420604417', 0.125, 0.125, 0.925, 'https://twitter.com/sbolusonsoz/status/1564926450096013313', 1.0, 'https://twitter.com/UYYAZAN/status/1597138789108895744', 0.125, 0.125, 0.925, 1.0, 0.125, 0.925, 0.925, 0.125, 1.0, 0.925, 1.0, 0.0, 0.125, 'https://twitter.com/SousaAngelene/status/1363226564683522051', 0.925, 0.925, 0.0, 1.0, 1.0, 0.125, 0.125, 0.125, 0.925, 1.0, 1.0, 0.925, 0.925, 0.925, 0.125, 0.925, 0.925, 0.925, 0.125, 0.125, 0.125, 'https://twitter.com/NiyaziSele/status/1446231430653255687', 0.925, 0.925, 1.0, 0.925, 0.125, 0.125, 'https://twitter.com/KAKISKALI/status/1370770949889490947', 0.925, 1.0, 0.125, 0.925, 0.125, 0.125, 1.0, 0.925, 1.0, 0.125, 1.0, 0.125, 0.125, 'https://twitter.com/avcisirvan/status/1575476973316579329', 0.125, 1.0, 0.925, 1.0, 0.925, 0.

### This part is important! We expect you to return your predictions in the following format:

In [500]:
modelPredTweet = dict([(x,y) for x,y in zip([*dfPolitical_test.tweet_id], predictions_political)])
modelPredTweet

{'1434787703783051264': 0.125,
 '1367571642604544000': 0.125,
 '1589993032975544320': 1.0,
 '1565312596135354373': 0.925,
 '1579558096833511424': 1.0,
 '1439547067337256967': 0.125,
 '1559963768372740098': 'https://twitter.com/awfhdsr6/status/1559963768372740098',
 '1562853131251118081': 'https://twitter.com/akanturker/status/1562853131251118081',
 '1586021183958704128': 0.925,
 '1585766233491886081': 0.925,
 '1427746815420604417': 'https://twitter.com/venxzhan/status/1427746815420604417',
 '1352635736537882629': 0.125,
 '1415032260571680768': 0.125,
 '1548636597628899328': 0.925,
 '1564926450096013313': 'https://twitter.com/sbolusonsoz/status/1564926450096013313',
 '1585634359612420101': 1.0,
 '1597138789108895744': 'https://twitter.com/UYYAZAN/status/1597138789108895744',
 '1391681495622995971': 0.125,
 '1389951943343316995': 0.125,
 '1452348722810138646': 0.925,
 '1595829502021623812': 1.0,
 '1413108476348354562': 0.125,
 '1579408398894137344': 0.925,
 '1570758749606019073': 0.925,


In [495]:
import operator

modelPredTweet = dict([(x,y) for x,y in zip([*dfPolitical_test.tweet_id], predictions_political)])

predictionAmounts = {}
predictionList = []

for key, value in modelPredTweet.items():
    if value[1] in predictionList:
        predictionAmounts[value[1]] += 1
    else:
        predictionList.append(value[1])
        predictionAmounts[value[1]] = 1
        
        
sortedPredictionAmounts = dict(sorted(predictionAmounts.items(), key=operator.itemgetter(1), reverse=True))

totalpoints = 0
for key, value in sortedPredictionAmounts.items():
    print("Prediction:", key, "->", value, "times")
    totalpoints += (key * value)

print(totalpoints/5000)

modelPredTweet

Prediction: 0.925 -> 1781 times
Prediction: 1.0 -> 1505 times
Prediction: 0.125 -> 1217 times
Prediction: 0.6710780613951839 -> 470 times
Prediction: 0.0 -> 27 times
0.7239913377711473


{'1434787703783051264': ['https://twitter.com/jerviajeon/status/1434787703783051264',
  0.125],
 '1367571642604544000': ['https://twitter.com/BakiDiren/status/1367571642604544000',
  0.125],
 '1589993032975544320': ['https://twitter.com/hasannegr/status/1589993032975544320',
  1.0],
 '1565312596135354373': ['https://twitter.com/Nursela60044416/status/1565312596135354373',
  0.925],
 '1579558096833511424': ['https://twitter.com/hiranur55508054/status/1579558096833511424',
  1.0],
 '1439547067337256967': ['https://twitter.com/_melek_3451/status/1439547067337256967',
  0.125],
 '1559963768372740098': ['https://twitter.com/awfhdsr6/status/1559963768372740098',
  0.6710780613951839],
 '1562853131251118081': ['https://twitter.com/akanturker/status/1562853131251118081',
  0.6710780613951839],
 '1586021183958704128': ['https://twitter.com/spytron/status/1586021183958704128',
  0.925],
 '1585766233491886081': ['https://twitter.com/ekalkanli2023/status/1585766233491886081',
  0.925],
 '142774681

## 3.2. Predictions for Users (Bot or Not)

In [490]:
evaluationUserDf = pd.read_csv('{}evaluation-round{}-user.csv'.format(PATH_TO_DOWNLOADED, ROUND), dtype={0: str}, header=None, names=['user_screen_name'])
evaluationUserDf = evaluationUserDf.dropna()
# merge it with the political dataframe so that you can use the make predictions based on the variables
dfBot_test = dfBotAll.merge(evaluationUserDf)

# define X as we did above in section (2.x.2. Separate X and y values)
X = dfBot_test[['follow_similarity', 'copy_tweets', 'avg_hashtag', 'follow_threshold', 'retweet_ratio']]
regression_bot = dtc_bot.predict_proba(X)

# make predictions based on these variables
predictions_bot = []
import numpy as np
from array import array

for index, row in dfBot_test.iterrows():
    prediction = regression_bot[index]
    predictions_bot.append(prediction[1])

print(predictions_bot)

[0.3024161413956731, 0.030215772190612018, 0.14201884404863976, 0.08320498951288835, 0.030763837387790916, 0.060076609685280766, 0.08077512251858512, 0.03129426583038309, 0.07839604534623909, 0.05692373282666738, 0.058667894962021835, 0.069933996405629, 0.03572937985252702, 0.05384780497215941, 0.03816562549564547, 0.07372748601041393, 0.1167063754975288, 0.24692369147333693, 0.09177276754066034, 0.03693988764958814, 0.09852061421426148, 0.06467706353004318, 0.32631115724214815, 0.10428432257850478, 0.06229622168413799, 0.26432806902119477, 0.08001990026725639, 0.19358454617388332, 0.06271371698670099, 0.1992870770732571, 0.07907718228477546, 0.09218722987219899, 0.0719384082157655, 0.0795060941288751, 0.04480381510187968, 0.16328651971591876, 0.05020794264172275, 0.09135248516038956, 0.17857361399530047, 0.05303071027806987, 0.13830825777068925, 0.2807800321406041, 0.09053538708349917, 0.08204695198272713, 0.04984033601869586, 0.03689497123550312, 0.10302855457358802, 0.08901558789956

In [491]:
modelPredUser = dict([(x,float(y)) for x,y in zip([*dfBot_test.user_screen_name], predictions_bot)])
modelPredUser

{'biologselim': 0.3024161413956731,
 'omerakdag34': 0.030215772190612018,
 'bilgin21604923': 0.14201884404863976,
 '_sydneycarton_': 0.08320498951288835,
 'denizlihabercom': 0.030763837387790916,
 'burakerbaychp': 0.060076609685280766,
 'mvnez': 0.08077512251858512,
 'qara118': 0.03129426583038309,
 'nabiyonyevrum': 0.07839604534623909,
 'farukhalit2': 0.05692373282666738,
 'harlunoshi': 0.058667894962021835,
 'heritagepaix': 0.069933996405629,
 'nuranwolf': 0.03572937985252702,
 'politikgundem': 0.05384780497215941,
 'isakethudax': 0.03816562549564547,
 'enveraysevera': 0.07372748601041393,
 'ilaydejaneiro': 0.1167063754975288,
 '1905anason': 0.24692369147333693,
 'eraydurgut03': 0.09177276754066034,
 'dasiskein': 0.03693988764958814,
 'ercan_bas29': 0.09852061421426148,
 'mett_1907': 0.06467706353004318,
 'ondemir066': 0.32631115724214815,
 'semihyeteer': 0.10428432257850478,
 'haberinyokcokk': 0.06229622168413799,
 'meleky_ozaydin': 0.26432806902119477,
 'mehmetaltay64': 0.080019900

# PREPARE SUBMISSION

You will need to submit exact same file produced by using the following code. Any deviation from the desired format willbe marked as 0.

In [501]:
# Explain your approach

data_explanations = '''
  For the model training I have used the given dataset, and added some extra data from our group's annotations.
  I have merged them with the corresponding JSON files to get desired features that I used in the training.
  I have analyzed all the tweets in the user_tweets JSON file, and created some political entity lists for detection.
'''

feature_explanations = '''
  I have created some features in order to detect if a tweet is a political tweet, or not:
  1) Number of political keywords: I created a huge list with many possible political keywords, and this feature calculates their count in the given tweet.
  2) Binarized number of political keywords: This feature specifies if the given tweet has some political keywords, or not.
  3) Number of political mentions: I created a huge list with many possible political users, and this feature calculates their count in the given tweet.
  4) Binarized number of political mentions: This feature specifies if the given tweet has some political mentions, or not.
  5) Number of political hashtags: I created a huge list with many possible political hashtags, and this feature calculates their count in the given tweet.
  6) Binarized number of political hashtags: This feature specifies if the given tweet has some political hashtags, or not.
  8) Politician bias elimination feature: This feature shows if the given tweet mentioning to an important politician: RTErdogan in this case.
  9) Net political feature: I created a tiny list of strings which implies being a political tweet for sure.
  10) No political feature: I created a tiny list of strings which implies being not a political tweet for sure.
  
  I have created some features in order to detect if a user is a bot, or not:
  1) Number of duplicate tweets: I calculated the approximate number of duplicate tweets of the user.
  2) Retweet ratio: This feature shows the ratio of retweets to all tweets of the user.
  3) Follow similarity: This feature specifies the similarity of followers and following counts.
  4) Average hashtag usage: Average number of hashtags used in all tweets of the user.
  5) Follow threshold: This features checks if the user is above a threshold, or not.
'''

model_explanations = '''
  I have used Logistic Regression, even though this is a classification problem, I calculated the probabilities.
  I have split the data into two train and test parts and then did cross validation.
  At the end, my regression models returns values between 0-1 for both isPolitical and isBot predictions.
'''

additional_explanations = '''
  In the project, I preferred to use binarized versions of some features since they have a higher accuracy.
  Also, I have made corrections for the predictions having some patterns in some features.
'''


In [502]:
predictions = {
    'round': ROUND,
    'student_id': STUDENT_ID,
    'user_predictions': modelPredUser,
    'tweet_predictions': modelPredTweet,
    'explanations': {
        'data': data_explanations,
        'feature': feature_explanations,
        'model': model_explanations,
        'other': additional_explanations,
    }
}

with open('predictions-{}_round{}.json'.format(STUDENT_ID, ROUND), 'w') as fl:
    fl.write(json.dumps(predictions, indent=4))

In [611]:
# Test your submission file

submission = json.load(open('predictions-{}_round{}.json'.format(STUDENT_ID, ROUND), 'r'))
submission

{'round': 2,
 'student_id': '28029',
 'user_predictions': {'nedenburdaysam': 0.0664559520520103,
  'biologselim': 0.3002476599788819,
  'bilgin21604923': 0.1420321337648567,
  'denizlihabercom': 0.030255195088112006,
  'burakerbaychp': 0.059374639770995837,
  'mustafaarst': 0.12610226114071782,
  'mvnez': 0.0809173090433058,
  'farukhalit2': 0.056468015724578165,
  'harlunoshi': 0.05839244865172983,
  'tamerduran_1': 0.12598383216529938,
  'donkisotumsu': 0.057761012374140296,
  'enveraysevera': 0.07299301749099911,
  'gendenmukatol': 0.11682381111376493,
  '1905anason': 0.24669103429958214,
  'dasiskein': 0.03689405994366066,
  'ercan_bas29': 0.09850794117920716,
  'mett_1907': 0.06509372353789405,
  'haberinyokcokk': 0.06220215658005473,
  'han34nesli': 0.06344154486013827,
  'mehmetaltay64': 0.07910104926897146,
  'nurtencam2': 0.19292217994250255,
  'berkeduranovic': 0.13655513766841815,
  'cagdasadim': 0.03788751082514288,
  'yorumsuzadam87': 0.19852953292663464,
  'twitsildiren':

In [613]:
VERSION = 1
zort = [0, 0, 0, 0]

with open('predictions-{}_round{}_updated1.json'.format(STUDENT_ID, ROUND, VERSION), 'r') as file:
    data = json.load(file)
    key_list = list(data['tweet_predictions'].keys())
    val_list = list(data['tweet_predictions'].values())
    
    for i in range(len(key_list)):
        if val_list[i] == 0.22077716093432836:
            data['tweet_predictions'][key_list[i]] = 0.42
            zort[0] += 1
        if val_list[i] == 0.8597141060057494:
            data['tweet_predictions'][key_list[i]] = 0.88
            zort[1] += 1
        if val_list[i] == 0.9323017039862496:
            data['tweet_predictions'][key_list[i]] = 0.95
            zort[2] += 1
        if val_list[i] == 1.0:
            zort[3] += 1
    # print(data)
print(zort)

[2345, 329, 192, 2122]


In [ ]:
old_list = [2345, 329, 192, 2122]
new_list = [2346, 305, 183, 2166]

In [589]:
zort = [0, 0, 0, 0]
with open('predictions-{}_round{}_updated{}.json'.format(STUDENT_ID, ROUND, VERSION+1), 'w') as file:
    key_list = list(data['tweet_predictions'].keys())
    val_list = list(data['tweet_predictions'].values())
    
    for i in range(len(key_list)):
        if val_list[i] == 0.44155432186865673:
            data['tweet_predictions'][key_list[i]] = 0.42
            zort[0] += 1
        if val_list[i] == 0.8847141060057493:
            data['tweet_predictions'][key_list[i]] = 0.88
            zort[1] += 1
        if val_list[i] == 0.9573017039862496:
            data['tweet_predictions'][key_list[i]] = 0.95
            zort[2] += 1
        if val_list[i] == 1.0:
            zort[3] += 1
            
    # file.write(json.dumps(data, indent=4))
print(zort)